# Generate a restricted SCvH EOS table for a H-He mixture
Generate a new EOS table based on the SCvH equation of state [Saumon et al. 1995](https://ui.adsabs.harvard.edu/abs/1995ApJS...99..713S/abstract) for Hydrogen and Helium with extensions to lower temperatures [Vazan et al. 2013](https://academic.oup.com/mnras/article/434/4/3283/961935) restricted to low densities and temperatures where the isotherms behave sensible.

### Import modules

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import interpolate as interp
#from scipy import optimize
import numpy as np

In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'retina'

In [ ]:
mpl.rcParams['figure.dpi'] = 100

In [ ]:
def read_eos_table_dt(filename, delimiter=None):
    data = np.loadtxt(filename, delimiter=delimiter) 

    logT_table   = data[:, 0]
    logrho_table = data[:, 1]
    logP_table   = data[:, 2]
    logu_table   = data[:, 3]
    logs_table   = data[:, 4]

    # All the SCvH EOS tables are tabulated along isotherms
    logT = np.unique(logT_table)
    nT = np.size(logT)

    # For the mixture tables the number of grid points in rho the same for each isotherm
    nRho = int(np.size(logT_table)/nT)
    
    print("nT = {:} nRho = {:}".format(nT, nRho))
    
    logrho = logrho_table[0:nRho]
    
    logrho_min = np.min(logrho)
    logrho_max = np.max(logrho)
    logT_min   = np.min(logT)
    logT_max   = np.max(logT)

    print("logrho_min = {:}".format(logrho_min))
    print("logrho_max = {:}".format(logrho_max))
    print("logT_min   = {:}".format(logT_min))
    print("logT_max   = {:}".format(logT_max))
    print()

    # Split into arrays of constant T
    logP_array = np.split(logP_table, nT)
    logu_array = np.split(logu_table, nT)
    logs_array = np.split(logs_table, nT)

    # Generate 2d arrays
    logP = np.vstack(logP_array)
    logu = np.vstack(logu_array)
    logs = np.vstack(logs_array)
      
    eos_table_dt = {
        "nT":         nT,
        "nRho":       nRho,
        "logT":       logT,
        "logrho":     logrho,
        "logrho_min": logrho_min,
        "logrho_max": logrho_max,
        "logT_min":   logT_min,
        "logT_max":   logT_max,
        "logP":       logP,
        "logu":       logu,
        "logs":       logs,
    }
    
    return eos_table_dt

In [ ]:
# Write an EOS table to an output file.
def write_eos_table(output_file, logrho_axis, logT_axis, logP_array, logu_array, logs_array, delimiter=" ", comments="#", input_file=""):
    nRho = np.size(logrho_axis)
    nT = np.size(logT_axis)
    
    # Store the different isotherms in 1d arrays
    logrho = np.tile(logrho_axis, nT)
    logT = np.repeat(logT_axis, nRho)

    logP = logP_array.flatten(order='C')
    logu = logu_array.flatten(order='C')
    logs = logs_array.flatten(order='C')
    
    header = " nT = {:} nRho= {:} (input file: {:})\n"\
             " {:} {:} {:} {:} {:}".format(nT, nRho, input_file, "logT [K]", "logRho [g/cc]", "logP [barye]", "logE [erg/g]", "logS [erg/g/K]")
    
    np.savetxt(output_file, np.column_stack([logT, logrho, logP, logu, logs]), header=header, fmt='%.8e', delimiter=delimiter, comments=comments)
    

In [ ]:
def press_ideal_gas(rho, T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/(mu*m_H)*rho*T

In [ ]:
def intenergy_ideal_gas(T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/((gamma-1.0)*mu*m_H)*T

In [ ]:
def entropy_ideal_gas(T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/(mu*m_H)*(np.log(T/T0) - 1.0/(gamma-1.0)*np.log(rho/rho0))

The mixture was obtained from the additive volume law (AVL) with Y=0.278 using bilinear spline interpolation and extrapolation.

In [ ]:
ext_table_file = "scvh_extended_dt_hydrogen_722_helium_278.data"
scvh_hhe_ext = read_eos_table_dt(ext_table_file)

In [ ]:
# Test if write_eos_table works
#write_eos_table("output.txt", scvh_hhe_ext['logrho'], scvh_hhe_ext['logT'], scvh_hhe_ext['logP'], scvh_hhe_ext['logu'], scvh_hhe_ext['logs'], input_file=ext_table_file)

In [ ]:
# Skip some curves otherwise the plots are very hard to read
nSkipT = 4
nSkipRho = 10

nSkipT = 1
nSkipRho = 1

In [ ]:
# Plot log(P), log(u) and log(s) of the original table
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# P(rho, T=const)
for i in range(0, scvh_hhe_ext['nT'], nSkipT):
    ax[0][0].plot(scvh_hhe_ext['logrho'], scvh_hhe_ext['logP'][i,:], '-')
    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, scvh_hhe_ext['nRho'], nSkipRho):
    ax[0][1].plot(scvh_hhe_ext['logT'], scvh_hhe_ext['logP'][:,i], '-')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, scvh_hhe_ext['nT'], nSkipT):
    ax[1][0].plot(scvh_hhe_ext['logrho'], scvh_hhe_ext['logu'][i,:], '-')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, scvh_hhe_ext['nRho'], nSkipRho):
    ax[1][1].plot(scvh_hhe_ext['logT'], scvh_hhe_ext['logu'][:,i], '-')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, scvh_hhe_ext['nT'], nSkipT):
    ax[2][0].plot(scvh_hhe_ext['logrho'], scvh_hhe_ext['logs'][i,:], '-')

ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, scvh_hhe_ext['nRho'], nSkipRho):
    ax[2][1].plot(scvh_hhe_ext['logT'], scvh_hhe_ext['logs'][:,i], '-')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
def generate_restricted_eos_table(scvh_eos_table, logrho_min, logrho_max, logT_min, logT_max):
    # Find all grid point that are in the range of the restricted EOS table
    index_logrho_grid = np.where((scvh_eos_table['logrho'] >= logrho_min) & (scvh_eos_table['logrho'] <= logrho_max))[0]
    index_logT_grid = np.where((scvh_eos_table['logT'] >= logT_min) & (scvh_eos_table['logT'] < logT_max))[0]

    nRho_grid = np.size(index_logrho_grid)
    nT_grid = np.size(index_logT_grid)
    print("nRho= {:} nT= {:}".format(nRho_grid, nT_grid))

    logrho_grid = scvh_eos_table['logrho'][index_logrho_grid]
    logT_grid = scvh_eos_table['logT'][index_logT_grid]
    
    index_logrho_grid_min = np.min(index_logrho_grid)
    index_logrho_grid_max = np.max(index_logrho_grid)  
    index_logT_grid_min = np.min(index_logT_grid)
    index_logT_grid_max = np.max(index_logT_grid)
    
    # Note that when slicing the last index is not included
    logP_grid = scvh_eos_table['logP'][index_logT_grid_min:index_logT_grid_max+1,index_logrho_grid_min:index_logrho_grid_max+1]
    logu_grid = scvh_eos_table['logu'][index_logT_grid_min:index_logT_grid_max+1,index_logrho_grid_min:index_logrho_grid_max+1]
    logs_grid = scvh_eos_table['logs'][index_logT_grid_min:index_logT_grid_max+1,index_logrho_grid_min:index_logrho_grid_max+1]

    scvh_eos_restricted = {
        "nT":         nT_grid,
        "nRho":       nRho_grid,
        "logT":       logT_grid,
        "logrho":     logrho_grid,
        "logrho_min": logrho_grid_min,
        "logrho_max": logrho_grid_max,
        "logT_min":   logT_grid_min,
        "logT_max":   logT_grid_max,
        "logP":       logP_grid,
        "logu":       logu_grid,
        "logs":       logs_grid,
    }
    
    return scvh_eos_restricted

In [ ]:
# Limit the data to the region in which we are interested
logrho_grid_min = -14.0
logrho_grid_max = -4.0

logT_grid_min = np.min(scvh_hhe_ext['logT'])
logT_grid_max = 3.46
logT_grid_max = 3.49

"""
print("logrho_grid_min= {:} rho_grid_min= {:}".format(logrho_grid_min, 10.0**logrho_grid_min))
print("logrho_grid_max= {:} rho_grid_max= {:}".format(logrho_grid_max, 10.0**logrho_grid_max))
print("logT_grid_min= {:} T_grid_min= {:}".format(logT_grid_min, 10.0**logT_grid_min))
print("logT_grid_max= {:} T_grid_max= {:}".format(logT_grid_max, 10.0**logT_grid_max))
print()
"""
scvh_hhe_rest = generate_restricted_eos_table(scvh_hhe_ext, logrho_grid_min, logrho_grid_max, logT_grid_min, logT_grid_max)

print("logrho_grid_min= {:} rho_grid_min= {:}".format(scvh_hhe_rest['logrho_min'], 10.0**scvh_hhe_rest['logrho_min']))
print("logrho_grid_max= {:} rho_grid_max= {:}".format(scvh_hhe_rest['logrho_max'], 10.0**scvh_hhe_rest['logrho_max']))
print("logT_grid_min= {:} T_grid_min= {:}".format(scvh_hhe_rest['logT_min'], logT_grid_min, 10.0**scvh_hhe_rest['logT_min']))
print("logT_grid_max= {:} T_grid_max= {:}".format(scvh_hhe_rest['logT_max'], 10.0**scvh_hhe_rest['logT_max']))
print()

# Write a new EOS table
write_eos_table("scvh_extended_dt_hydrogen_722_helium_278_lowrhot.txt", scvh_hhe_rest['logrho'], scvh_hhe_rest['logT'], scvh_hhe_rest['logP'], scvh_hhe_rest['logu'], scvh_hhe_rest['logs'], input_file=ext_table_file)

# Now load the new EOS table to check if it was properly written
scvh_hhe_rest = read_eos_table_dt("scvh_extended_dt_hydrogen_722_helium_278_lowrhot.txt")

In [ ]:
# Skip some curves otherwise the plots are very hard to read
nSkipT = 3
nSkipRho = 10

In [ ]:
# Plot the restricted grid and check that logP, logu and logs agree with logrho and logT
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

fig.suptitle(r"{:} $\leq \log(\rho) \leq$ {:}, {:} $\leq \log(T) \leq$ {:}".format(scvh_hhe_rest['logrho_min'], scvh_hhe_rest['logrho_max'], scvh_hhe_rest['logT_min'], scvh_hhe_rest['logT_max']), x=0.5, y=0.0, verticalalignment='bottom', horizontalalignment='center')

# Generate interpolation functions
logP_ext_int = interp.interp2d(scvh_hhe_ext['logrho'], scvh_hhe_ext['logT'], scvh_hhe_ext['logP'], kind='linear')
logu_ext_int = interp.interp2d(scvh_hhe_ext['logrho'], scvh_hhe_ext['logT'], scvh_hhe_ext['logu'], kind='linear')
logs_ext_int = interp.interp2d(scvh_hhe_ext['logrho'], scvh_hhe_ext['logT'], scvh_hhe_ext['logs'], kind='linear')

# P(rho, T=const)
for i in range(0, scvh_hhe_rest['nT'], nSkipT):
    ax[0][0].plot(scvh_hhe_rest['logrho'], scvh_hhe_rest['logP'][i,:], '-')
    ax[0][0].plot(scvh_hhe_rest['logrho'], logP_ext_int(scvh_hhe_rest['logrho'], scvh_hhe_rest['logT'][i]), '--')

    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, scvh_hhe_rest['nRho'], nSkipRho):
    ax[0][1].plot(scvh_hhe_rest['logT'], scvh_hhe_rest['logP'][:,i], '-')
    ax[0][1].plot(scvh_hhe_rest['logT'], logP_ext_int(scvh_hhe_rest['logrho'][i], scvh_hhe_rest['logT']), '--')


ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, scvh_hhe_rest['nT'], nSkipT):
    ax[1][0].plot(scvh_hhe_rest['logrho'], scvh_hhe_rest['logu'][i,:], '-')
    ax[1][0].plot(scvh_hhe_rest['logrho'], logu_ext_int(scvh_hhe_rest['logrho'], scvh_hhe_rest['logT'][i]), '--')


ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, scvh_hhe_rest['nRho'], nSkipRho):
    ax[1][1].plot(scvh_hhe_rest['logT'], scvh_hhe_rest['logu'][:,i], '-')
    ax[1][1].plot(scvh_hhe_rest['logT'], logu_ext_int(scvh_hhe_rest['logrho'][i], scvh_hhe_rest['logT']), '--')


ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, scvh_hhe_rest['nT'], nSkipT):
    ax[2][0].plot(scvh_hhe_rest['logrho'], scvh_hhe_rest['logs'][i,:], '-')
    ax[2][0].plot(scvh_hhe_rest['logrho'], logs_ext_int(scvh_hhe_rest['logrho'], scvh_hhe_rest['logT'][i]), '--')


ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, scvh_hhe_rest['nRho'], nSkipRho):
    ax[2][1].plot(scvh_hhe_rest['logT'], scvh_hhe_rest['logs'][:,i], '-')
    ax[2][1].plot(scvh_hhe_rest['logT'], logs_ext_int(scvh_hhe_rest['logrho'][i], scvh_hhe_rest['logT']), '--')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
# Skip some curves otherwise the plots are very hard to read
nSkipT = 3
nSkipRho = 10

In [ ]:
# Compare the reduced grid to an ideal gas
fig, ax = plt.subplots(2, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 2*y)

# P(rho, T=const)
ax[0][0].set_prop_cycle(None)
for i in range(0, scvh_hhe_rest['nT'], nSkipT):
    ax[0][0].plot(scvh_hhe_rest['logrho'], scvh_hhe_rest['logP'][i,:], '-')

ax[0][0].set_prop_cycle(None)
for i in range(0, scvh_hhe_rest['nT'], nSkipT):
    ax[0][0].plot(scvh_hhe_rest['logrho'], np.log10(press_ideal_gas(10.0**scvh_hhe_rest['logrho'], 10.0**scvh_hhe_rest['logT'][i])), '--')

ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
ax[0][1].set_prop_cycle(None)
for i in range(0, scvh_hhe_rest['nRho'], nSkipRho):
    ax[0][1].plot(scvh_hhe_rest['logT'], scvh_hhe_rest['logP'][:,i], '-')

ax[0][1].set_prop_cycle(None)
for i in range(0, scvh_hhe_rest['nRho'], nSkipRho):
    ax[0][1].plot(scvh_hhe_rest['logT'], np.log10(press_ideal_gas(10.0**scvh_hhe_rest['logrho'][i], 10.0**scvh_hhe_rest['logT'])), '--')

    
ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
ax[1][0].set_prop_cycle(None)
for i in range(0, scvh_hhe_rest['nT'], nSkipT):
    ax[1][0].plot(scvh_hhe_rest['logrho'], scvh_hhe_rest['logu'][i,:], '-')

ax[1][0].set_prop_cycle(None)
for i in range(0, scvh_hhe_rest['nT'], nSkipT):
    ax[1][0].plot(scvh_hhe_rest['logrho'], np.ones(scvh_hhe_rest['nRho'])*np.log10(intenergy_ideal_gas(10.0**scvh_hhe_rest['logT'][i])), '--')

        #ax[1][0].plot(logrho_grid, *np.log10(intenergy_ideal_gas(10**logT_grid[i])), '--')

    
ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")
ax[1][0].set(title="SCvH EOS H (T=const.)")


# u(rho=const, T)
ax[1][1].set_prop_cycle(None)
for i in range(0, scvh_hhe_rest['nRho'], nSkipRho):
    ax[1][1].plot(scvh_hhe_rest['logT'], scvh_hhe_rest['logu'][:,i], '-')

ax[1][1].set_prop_cycle(None)
for i in range(0, scvh_hhe_rest['nRho'], nSkipRho):
    ax[1][1].plot(scvh_hhe_rest['logT'], np.log10(intenergy_ideal_gas(10**scvh_hhe_rest['logT'])), '--')

ax[1][1].get_xlim()
ax[1][1].get_ylim()
